<a href="https://colab.research.google.com/github/nizam0768/models/blob/master/PyTorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
x = torch.rand(5,3)
print(x)

tensor([[0.1157, 0.1494, 0.7264],
        [0.3986, 0.8646, 0.0862],
        [0.6391, 0.1964, 0.8019],
        [0.4361, 0.0436, 0.8035],
        [0.9046, 0.4151, 0.1008]])


In [6]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

In [9]:
# Download training data from open datasets
training_data = datasets.FashionMNIST(
    root ="data",
    train = True,
    download = True,
    transform=ToTensor(),
)
# Download test data from open datasets

test_data = datasets.FashionMNIST(
    root = "data",
    train = False,
    download = True,
    transform=ToTensor(),
)

In [11]:
batch_size = 64
# Create data loaders
train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

for X, y in test_dataloader:
  print(f"Shape of X [N, C, H, W]: {X.shape}")
  print(f"Shape of y: {y.shape} {y.dtype}")
  break

Shape of X [N, C, H, W]: torch.Size([64, 1, 28, 28])
Shape of y: torch.Size([64]) torch.int64


In [12]:
device = torch.accelerator.current_accelerator().type if torch.accelerator.is_available() else "cpu"

In [13]:
print(f"Using {device} device")

Using cpu device


In [15]:
# Define model
class NeuralNetwork(nn.Module):
  def __init__(self):
    super().__init__()
    self.flatten = nn.Flatten()
    self.linear_relu_stack = nn.Sequential(
        nn.Linear(28*28, 512),
        nn.ReLU(),
        nn.Linear(512, 512),
        nn.ReLU(),
        nn.Linear(512, 10)
    )
  def forward(self, x):
    x = self.flatten(x)
    logits = self.linear_relu_stack(x)
    return logits



In [16]:
model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [17]:
loss_fn = nn.CrossEntropyLoss()

In [19]:
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

In [33]:
def train(dataloader, model, loss_fn, optimizer):
  size = len(dataloader.dataset)
  model.train()
  for batch, (X, y) in enumerate(dataloader):
    X, y = X.to(device), y.to(device)

    # compute pridiction error
    pred = model(X)
    loss = loss_fn(pred, y)

    # Backpropagation
    loss.backward()
    optimizer.step()
    optimizer.zero_grad()

    if batch % 100 == 0:
      loss, current = loss.item(), (batch + 1) * len(X)
      print(f"loss: {loss:>7f} [{current:>5d}/{size:>5d}]")


In [39]:
def test(dataloader, model, loss_fn):
  size = len(dataloader.dataset)
  num_batches = len(dataloader)
  model.eval()
  test_loss, correct = 0,0
  with torch.no_grad():
    for X, y in dataloader:
      X, y = X.to(device), y.to(device)
      pred = model(X)
      test_loss += loss_fn(pred, y).item()
      correct += (pred.argmax(1) == y).type(torch.float).sum().item()
  test_loss /= num_batches
  correct /= size
  print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:8f} \n")

In [40]:
epochs = 5
for t in range(epochs):
  print(f"Epoch {t+1} \n---------------------")
  train(train_dataloader, model, loss_fn, optimizer)
  test(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1 
---------------------
loss: 1.951303 [   64/60000]
loss: 1.912052 [ 6464/60000]
loss: 1.802848 [12864/60000]
loss: 1.830107 [19264/60000]
loss: 1.728087 [25664/60000]
loss: 1.685948 [32064/60000]
loss: 1.709407 [38464/60000]
loss: 1.615527 [44864/60000]
loss: 1.635894 [51264/60000]
loss: 1.532206 [57664/60000]
Test Error: 
 Accuracy: 58.6%, Avg loss: 1.541673 

Epoch 2 
---------------------
loss: 1.619678 [   64/60000]
loss: 1.572291 [ 6464/60000]
loss: 1.429465 [12864/60000]
loss: 1.491513 [19264/60000]
loss: 1.373578 [25664/60000]
loss: 1.367193 [32064/60000]
loss: 1.383939 [38464/60000]
loss: 1.313577 [44864/60000]
loss: 1.345613 [51264/60000]
loss: 1.244476 [57664/60000]
Test Error: 
 Accuracy: 62.4%, Avg loss: 1.267993 

Epoch 3 
---------------------
loss: 1.354787 [   64/60000]
loss: 1.325574 [ 6464/60000]
loss: 1.168738 [12864/60000]
loss: 1.267417 [19264/60000]
loss: 1.141392 [25664/60000]
loss: 1.162096 [32064/60000]
loss: 1.184479 [38464/60000]
loss: 1.130065 [4486

In [41]:
torch.save(model.state_dict(), "model.pth")
print("Saved PyTorch Model State to model.ptg")

Saved PyTorch Model State to model.ptg


In [42]:
model = NeuralNetwork().to(device)

In [43]:
model.load_state_dict(torch.load("model.pth", weights_only =True))

<All keys matched successfully>

In [44]:
classes = [
    "T-shirt/top",
    "Trouser",
    "Pullover",
    "Dress",
    "Coat",
    "Sandal",
    "Shirt",
    "Sneaker",
    "Bag",
    "Ankle boot",
]
model.eval()
x, y = test_data[0][0], test_data[0][1]
with torch.no_grad():
  pred = model(x)
  predicted, actual = classes[pred[0].argmax(0)], classes[y]
  print(f'Predicted:"{predicted}", Actual: "{actual}"')

Predicted:"Ankle boot", Actual: "Ankle boot"
